# Fine tune Mistral-Nemo-Instruct-2407 model using Axolotl framework

How to install dependencies (in HPC environment):

- load Python and cuDNN modules
- create a Python venv and activate it
- install dependencies from requirements.txt (e.g. torch)
- install Axolotl from git clone (pip won't work, see [this issue](https://github.com/OpenAccess-AI-Collective/axolotl/issues/945)):

```
git clone git@github.com:OpenAccess-AI-Collective/axolotl.git
cd axolotl
pip install -e '.[flash-attn,deepspeed]'
```


In [1]:
# Check if GPU is available
import torch
print('GPU available?', torch.cuda.is_available())
print('BF16 is supported?', torch.cuda.is_bf16_supported())

GPU available? True
BF16 is supported? True


In [2]:
# set model name etc.

MODEL_NAME = "mistralai/Mistral-Nemo-Instruct-2407"
MODEL_SHORT_NAME = MODEL_NAME.split('/')[-1]
SUFFIX = "FinGreyLit"
#SLICE = 1

In [3]:
# Load and prepare fine-tuning dataset

import json
import glob
import random

random.seed(42)  # for deterministic sampling of test set

train_files = glob.glob("../../llm-dataset/*-train.jsonl")
test_files = glob.glob("../../llm-dataset/*-test.jsonl")

EVAL_SIZE = 32  # how many documents to evaluate (i.e. calculate loss) on during fine-tuning
SYSTEM_PROMPT = "You are a skilled librarian specialized in meticulous cataloguing of digital documents."
INSTRUCTION = "Extract metadata from this document. Return as JSON."

def preprocess_sample(sample):
    output = json.dumps(sample["ground_truth"])
    input_ = json.dumps(sample["content"])
    # ShareGPT format
    conversations = [
        {'from': 'system', 'value': SYSTEM_PROMPT},
        {'from': 'user', 'value': INSTRUCTION + "\n\n" + input_},
        {'from': 'gpt', 'value': output}
    ]
    return {"conversations": conversations}

def dataset_to_records(files):
    records = []
    for filename in files:
        with open(filename) as infile:
            for line in infile:
                sample = json.loads(line)
                records.append(preprocess_sample(sample))
    return records

def write_jsonl(records, filename):
    with open(filename, "w") as outfile:
        for record in records:
            json.dump(record, outfile)
            outfile.write("\n")

train_recs = dataset_to_records(train_files)
random.shuffle(train_recs)
write_jsonl(train_recs, "axolotl-train.jsonl")
print(f"Wrote {len(train_recs)} train records")

test_recs = dataset_to_records(test_files)
write_jsonl(test_recs, "axolotl-test.jsonl")
print(f"Wrote {len(test_recs)} test records")

eval_recs = random.sample(test_recs, EVAL_SIZE)
write_jsonl(eval_recs, "axolotl-eval.jsonl")
print(f"Wrote {len(eval_recs)} eval records")

Wrote 640 train records
Wrote 182 test records
Wrote 32 eval records


In [4]:
# Create Axolotl configuration file

CONFIG_FILE = f"config-{MODEL_SHORT_NAME}.yml"


CONFIG = f"""
base_model: {MODEL_NAME}
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer

load_in_8bit: false
load_in_4bit: false
strict: false

datasets:
  - path: axolotl-train.jsonl
    ds_type: json
    split: train
    type: sharegpt
    conversation: chatml

test_datasets:
  - path: axolotl-eval.jsonl
    ds_type: json
    split: train
    type: sharegpt
    conversation: chatml

output_dir: ./out-{MODEL_SHORT_NAME}

#chat_template: chatml

peft_use_dora: true
adapter: lora
lora_r: 16
lora_alpha: 32
lora_dropout: 0.05
lora_target_linear: true

sequence_len: 4096
sample_packing: true
eval_sample_packing: false
pad_to_sequence_len: true

wandb_project:
wandb_entity:
wandb_watch:
wandb_name:
wandb_log_model:

gradient_accumulation_steps: 4
micro_batch_size: 2
eval_batch_size: 2
num_epochs: 5
optimizer: adamw_bnb_8bit
lr_scheduler: cosine
learning_rate: 0.0002

train_on_inputs: false
group_by_length: false
bf16: true
fp16: false
tf32: false

gradient_checkpointing: true  # true: saves VRAM but is slower to train
early_stopping_patience:
resume_from_checkpoint:
local_rank:
logging_steps: 1
xformers_attention:
flash_attention: true

warmup_steps: 10
evals_per_epoch: 2
eval_table_size:
eval_table_max_new_tokens: 128
saves_per_epoch: 1
debug:
weight_decay: 0.0
fsdp:
fsdp_config:
special_tokens:
  pad_token: "<pad>"

""".strip()

with open(CONFIG_FILE, 'w') as outfile:
    print(CONFIG, file=outfile)

In [5]:
%%time

!venv/bin/accelerate launch -m axolotl.cli.train {CONFIG_FILE}

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[2025-09-25 15:04:29,321] [INFO] [datasets.<module>:58] [PID:1652969] PyTorch version 2.3.1 available.
[2025-09-25 15:04:42,579] [INFO] [axolotl.utils.config.models.input.check_eval_packing:958] [PID:1652969] [RANK:0] setting `remove_unused_columns: false` for when sample_packing and eval_sample_packing don't match
[2025-09-25 15:04:42,580] [WARNING] [axolotl.utils.config.models.input.hint_lora_8bit:1044] [PID:1652969] [RANK:0] We recommend setting `load_in_8bit: true` for LORA finetuning
[2025-09-25 15:04:42,788] [INFO] [axolotl.normalize_config:183] [PID:1652969] [RANK:0] GPU memory usage baseline: 0.000GB (

# Use the fine-tuned model

In [6]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

qlora_model = f"./out-{MODEL_SHORT_NAME}"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False, padding_side='left')
tokenizer.add_special_tokens({'pad_token': '<pad>'})
base_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto", torch_dtype=torch.float16, attn_implementation="flash_attention_2").eval()
model = PeftModel.from_pretrained(base_model, qlora_model)


/wrk-vakka/users/oisuomin/git/FinGreyLit/experiments/axolotl-finetune-llm/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 5/5 [06:57<00:00, 83.47s/it]


In [7]:
# push the LoRA model (PEFT adapter) to HF Hub

#hub_model_id = f"NatLibFi/{MODEL_SHORT_NAME}-{SUFFIX}"
#model.push_to_hub(hub_model_id)

In [8]:
# merge the LoRA into the base model for inference
model = model.merge_and_unload()

In [9]:
# Save the merged model to a directory (along with tokenizer)

merged_model_dir = f"merged-{MODEL_SHORT_NAME}-{SUFFIX}"
model.save_pretrained(merged_model_dir)
tokenizer.save_pretrained(merged_model_dir)


('merged-Mistral-Nemo-Instruct-2407-FinGreyLit/tokenizer_config.json',
 'merged-Mistral-Nemo-Instruct-2407-FinGreyLit/special_tokens_map.json',
 'merged-Mistral-Nemo-Instruct-2407-FinGreyLit/tokenizer.json')

In [10]:
MAX_BATCHED_LENGTH = 4096

def generate(messages_batch):
    texts = tokenizer.apply_chat_template(messages_batch, add_generation_prompt=True, tokenize=False)
    inputs = tokenizer(texts, padding="longest", return_tensors="pt", return_token_type_ids=False)
    inputs = {key: val.cuda() for key, val in inputs.items()}
    if len(messages_batch) > 1 and inputs['input_ids'].shape[1] > MAX_BATCHED_LENGTH:
        # there are long documents in the batch - break it down to two smaller batches to avoid excessive padding and related problems
        half = int(len(messages_batch) / 2)
        return generate(messages_batch[:half]) + generate(messages_batch[half:])
    temp_texts=tokenizer.batch_decode(inputs["input_ids"], skip_special_tokens=True)

    gen_tokens = model.generate(
        **inputs,
        max_new_tokens=512,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    gen_text = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)
    return [i[len(temp_texts[idx]):] for idx, i in enumerate(gen_text)]


In [11]:
%%time

import json
from itertools import batched

BATCH_SIZE = 8  # requires ~70GB VRAM

def rec_to_messages(rec):
    return [
        {"role": msg["from"], "content": msg["value"]}
        for msg in rec["conversations"]
        if msg["from"] != "gpt"
    ]

# read the eval records from file
test_recs = []
with open("axolotl-test.jsonl") as testfile:
    for line in testfile:
        test_recs.append(json.loads(line))

#with open(f'test-records-{MODEL_SHORT_NAME}-slice{SLICE}.jsonl', 'w') as outfile:
with open(f'test-records-{MODEL_SHORT_NAME}.jsonl', 'w') as outfile:

    for batchno, rec_batch in enumerate(batched(test_recs, BATCH_SIZE)):
        messages_batch = [rec_to_messages(rec) for rec in rec_batch]
        responses = generate(messages_batch)
        gt_batch = [rec['conversations'][-1]["value"] for rec in rec_batch]

        for ground_truth, response in zip(gt_batch, responses):
            print(100 * "-")
            print("Ground Truth:")
            print(ground_truth)
            print("Prediction:")
            print(response)

            ground_truth = json.loads(ground_truth)

            try:
                prediction = json.loads(response)
            except json.JSONDecodeError:
                prediction = {}
        
            # rowid is set to unknown as we've lost it somewhere along the way...
            record = {"ground_truth": ground_truth, "prediction": prediction, "rowid": "unknown"}
            json.dump(record, outfile)
            outfile.write("\n")

----------------------------------------------------------------------------------------------------
Ground Truth:
{"language": "en", "title": "A light enterprise information security architecture model for creating and improving security architecture", "alt_title": ["Kevyt yritystietoturva-arkkitehtuurimalli tietoturva-arkkitehtuurin luomiseksi ja kehitt\u00e4miseksi {fi}"], "creator": ["Kossila, Johannes"], "year": "2019", "publisher": ["University of Turku"], "type_coar": "master thesis"}
Prediction:
{"language": "en", "title": "A light enterprise information security architecture model for creating and improving security architecture", "creator": ["Kossila, Johannes"], "year": "2019", "publisher": ["Turku School of Economics, University of Turku"], "type_coar": "master thesis"}
----------------------------------------------------------------------------------------------------
Ground Truth:
{"language": "en", "title": "An analysis of Facebook posts", "creator": ["To, My"], "year": 

In [12]:
# Analyze the statistics of the extracted metadata and save to file

import sys
sys.path.append('..')

from eval import MetadataEvaluator

#evaluator = MetadataEvaluator(f'test-records-{MODEL_SHORT_NAME}-slice{SLICE}.jsonl')
evaluator = MetadataEvaluator(f'test-records-{MODEL_SHORT_NAME}.jsonl')
results = evaluator.evaluate_records() #prediction_records[:9])

#statistics_filename = f'../results-axolotl-{MODEL_SHORT_NAME}-slice{SLICE}.md'
statistics_filename = f'../results-axolotl-{MODEL_SHORT_NAME}.md'
evaluator.save_md(results, statistics_filename)

In [ ]:
%%time
# convert the merged model to GGUF using llama.cpp tools (installed separately)

LLAMA_CPP_PATH = "../../../llama.cpp"
merged_model_dir = f"merged-{MODEL_SHORT_NAME}"

!{LLAMA_CPP_PATH}/venv/bin/python {LLAMA_CPP_PATH}/convert_hf_to_gguf.py {merged_model_dir} --outfile {MODEL_SHORT_NAME}-{SUFFIX}-f16.gguf

In [ ]:
%%time
# Quantize the F16 GGUF model to the 5+ bit Q5_K_M format
QTYPE = "Q5_K_M"

!{LLAMA_CPP_PATH}/llama-quantize {MODEL_SHORT_NAME}-{SUFFIX}-f16.gguf {MODEL_SHORT_NAME}-{SUFFIX}-{QTYPE}.gguf {QTYPE}